#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **AMPcamp First Notebook: Data Exploration Using Spark**
#### This notebook will implement the first of AMPcamp 5 from (http://ampcamp.berkeley.edu/5/exercises/data-exploration-using-spark.html)

<b>1.</b> Warm up by creating an RDD (Resilient Distributed Dataset) named pagecounts from the input files. In the Spark shell, the SparkContext is already created for you as variable sc.

In [ ]:
sc

In [ ]:
pagecounts = sc.textFile("data/pagecounts")
pagecounts

<b>2.</b> Let's take a peek a the data. You can use the take operation to get the first K record. Here K = 10.

In [ ]:
pagecounts.take(10)

Unfortunately this is not very readable because take() returns an array and Scala simply prints the array with each element separated by a comma. We can make it prettier by traversing the array to print each record on its own line.

In [ ]:
for x in pagecounts.take(10):
    print x

<b>3.</b> Let's see how many records in total are in this data set (this command will take a while, so read ahread while it is running).

In [ ]:
pagecounts.count()

This should launch 2 tasks on the your Spark cluster. If you look closely at the terminal, the console log is pretty chatty and tells you the progress of the tasks.

While it’s running, you can open the Spark web console to see the progress. To do this, open your favorite browser, and type in the following URL.

http://localhost:4040

Note that this page is only available if you have an active job or Spark shell.
You should see the Spark application status web interface, similar to the following:

![spark stage exampe](http://ampcamp.berkeley.edu/5/exercises/img/application-webui640.png)

The links in this interface allow you to track the job’s progress and various metrics about its execution, including task durations and cache statistics.

When your query finishes running, it should return the following count:

>1398882

<b>4.</b> Recall from above when we described the format of the data set, that the second field is the “project code” and contains information about the language of the pages. For example, the project code “en” indicates an English page. Let’s derive an RDD containing only English pages from pagecounts. This can be done by applying a filter function to pagecounts. For each record, we can split it by the field delimiter (i.e. a space) and get the second field-– and then compare it with the string “en”.

To avoid reading from disks each time we perform any operations on the RDD, we also <b>cache the RDD into memory</b>. This is where Spark really starts to to shine.

In [ ]:
enPages = pagecounts.filter(lambda x: x.split(" ")[1] == "en").cache()

<b>5.</b> How many records are there for English pages?

In [ ]:
enPages.count()

The first time this command is run, similar to the last count we did, it will take 2 - 3 minutes while Spark scans through the entire data set on disk. But since enPages was marked as “cached” in the previous step, if you run count on the same RDD again, it should return an order of magnitude faster.

If you examine the console log closely, you will see lines like this, indicating some data was added to the cache:
>13/02/05 20:29:01 INFO storage.BlockManagerMasterActor$BlockManagerInfo: Added rdd_2_172 in memory on ip-10-188-18-127.ec2.internal:42068 (size: 271.8 MB, free: 5.5 GB)

<b>6.</b> Let’s try something fancier. Generate a histogram of total page views on Wikipedia English pages for the date range represented in our dataset (May 5 to May 7, 2009). The high level idea of what we’ll be doing is as follows. First, we generate a key value pair for each line; the key is the date (the first eight characters of the first field), and the value is the number of pageviews for that date (the fourth field).

In [ ]:
enTuples = enPages.map(lambda x: x.split(" "))
enKeyValuePairs = enTuples.map(lambda x: (x[0][:8], int(x[3])))

Next, we shuffle the data and group all values of the same key together. Finally we sum up the values for each key. There is a convenient method called reduceByKey in Spark for exactly this pattern. Note that the second argument to reduceByKey determines the number of reducers to use. By default, Spark assumes that the reduce function is commutative and associative and applies combiners on the mapper side. Since we know there is a very limited number of keys in this case (because there are only 3 unique dates in our data set), let’s use only one reducer.

In [ ]:
from operator import add
enKeyValuePairs.reduceByKey(add, 1).collect()

The collect method at the end converts the result from an RDD to an array.

We can combine the previous three commands into one:

In [ ]:
enPages.map(lambda x: x.split(" ")).map(lambda x: (x[0][:8], int(x[3]))).reduceByKey(add).collect()

<b>7.</b> Suppose we want to find pages that were viewed more than 200,000 times during the three days covered by our dataset. Conceptually, this task is similar to the previous query. But, given the large number of pages (23 million distinct page names), the new task is very expensive. We are doing an expensive group-by with a lot of network shuffling of data.

To recap, first we split each line of data into its respective fields. Next, we extract the fields for page name and number of page views. We reduce by key again, this time with 40 reducers. Then we filter out pages with less than 200,000 total views over our time window represented by our dataset.

In [ ]:
(enPages.map(lambda x: x.split(" "))
        .map(lambda x: (x[2], int(x[3])))
        .reduceByKey(add, 40)
        .filter(lambda x: x[1] > 200000)
        .map(lambda x: (x[1], x[0]))
        .collect())

There is no hard and fast way to calculate the optimal number of reducers for a given problem; you will build up intuition over time by experimenting with different values.

<b>8.</b> You can explore the full RDD API by browsing the [Java/Scala](http://www.cs.berkeley.edu/~pwendell/strataconf/api/core/index.html#spark.RDD) or [Python](http://www.cs.berkeley.edu/~pwendell/strataconf/api/pyspark/index.html) API docs.



##Running Standalone Spark Programs
Because of time constraints, in this tutorial we focus on ad-hoc style analytics using the Spark shell. However, for many tasks, it makes more sense to write a standalone Spark program. We will return to this in the section on Spark Streaming below, where you will actually write a standalone Spark Streaming job. We aren’t going to cover how to structure, build, and run standalone Spark jobs here, but before we move on, we list here a few resources about standalone Spark jobs for you to come back and explore later.

First, on the AMI for this tutorial we have included “template” projects for Scala and Java standalone programs for both Spark and Spark streaming. The Spark ones can be found in the /root/scala-app-template and /root/java-app-template directories (we will discuss the Streaming ones later). Feel free to browse through the contents of those directories. You can also find examples of building and running Spark standalone jobs in [Java](http://spark.apache.org/docs/latest/quick-start.html#a-standalone-job-in-java) and in Scala as part of the Spark Quick Start Guide. For even more details, see Matei Zaharia’s [slides](http://ampcamp.berkeley.edu/wp-content/uploads/2012/06/matei-zaharia-part-2-amp-camp-2012-standalone-programs.pdf) and [talk video](http://www.youtube.com/watch?v=7k4yDKBYOcw&t=59m37s) about Standalone Spark jobs at the [first AMP Camp](http://ampcamp.berkeley.edu/amp-camp-one-berkeley-2012/).